# Rocking Curves

## calc rocking curve with xrt

In [ ]:
import numpy as np
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt
import xrt.backends.raycing.materials as rm
import xrt.backends.raycing.materials_crystals as rmc

crystal = rmc.Si(hkl=(2, 2, 0))
E = 33300
# crystal, E = rm.CrystalFromCell(xtl), 33300
dtheta = np.linspace(-30, 40, 601)
dt = dtheta[1] - dtheta[0]
theta = crystal.get_Bragg_angle(E) + dtheta*1e-6
refl = np.abs(crystal.get_amplitude(E, np.sin(theta))[0])**2  # s-polarization
#rc = np.convolve(refl, refl, 'same') / (refl.sum()*dt) * dt
refl_sq = refl*refl
rc = np.convolve(refl_sq, refl, 'same') / refl_sq.sum()
spline = UnivariateSpline(dtheta, rc-rc.max()/2, s=0)
r1, r2 = spline.roots()  # find the roots

plt.plot(dtheta, refl, 'r', label=u'one crystal\nFWHM = {0:.1f} µrad'.format(
    crystal.get_Darwin_width(E)*1e6))
plt.plot(dtheta, rc, 'b', label=u'two crystal\n(convolution)'
         u'\nFWHM = {0:.1f} µrad'.format(r2-r1))
plt.gca().set_xlabel(u'$\\theta - \\theta_{B}$ (µrad)')
plt.gca().set_ylabel(r'reflectivity')
plt.axvspan(r1, r2, facecolor='g', alpha=0.2)
plt.legend(loc='upper right', fontsize=12)

text = u'Rocking curve of {0}{1[0]}{1[1]}{1[2]} at E={2:.0f} eV'.format(
    crystal.name, crystal.hkl, E)
plt.text(0.5, 1.02, text, transform=plt.gca().transAxes, size=15, ha='center')
plt.show()

with open("Rocking_curve_XRT_33_3.txt", 'w') as f:
    for i in range(len(rc)):
        f.write(str(dtheta[i]) + ",     " + str(rc[i]) + "\n")

## Peak fitting

## measured

In [ ]:
import pandas as pd
flat_data = pd.read_csv("flat.dat", header=None)
plt.plot(flat_data[0], flat_data[1], 'bo')
plt.show()

In [ ]:
from scipy.optimize import curve_fit
from numpy import exp, pi


def gauss(x, H, A, x0, sigma):
    y = H + A * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))
    return y


def psvoigt(x,y0,A,x0,WL,WG,mu):

    return y0 + A*((mu*WL**2)/((x-x0)**2+WL**2)
                   + (1-mu)*exp(-(x - x0) ** 2 / (2 * WG ** 2)))


# unmodified
xdata = flat_data[0]
ydata = flat_data[1]
# use results below to scale data
# xdata = np.linspace(-5.230154*2, 5.230154*2, len(flat_data[0]))
# ydata = flat_data[1] * 0.3855519133586996

parameters, covariance = curve_fit(gauss, xdata, ydata, maxfev=10000)


fit_y = gauss(xdata, *parameters)
plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_y, '-', label='fit')
plt.legend()


In [ ]:
print(parameters)

## calculated

In [ ]:
calc_data = pd.read_csv("Rocking_curve_XRT_33_3.txt", header=None)
calc_data
# plt.plot(calc_data[0], calc_data[1], 'bo')
# plt.show()

In [ ]:
xdata2 = calc_data[0]
ydata2 = calc_data[1]
parameters2, covariance2 = curve_fit(gauss, xdata2, ydata2)

fit_y2 = gauss(xdata2, *parameters2)
plt.plot(xdata2, ydata2, 'o', label='data')
plt.plot(xdata2, fit_y2, '-', label='fit')
plt.legend()

In [ ]:
print(parameters2)

### compare results
Calculation has units of radian vs reflectivity and calculated are in usingts of mm vs voltage

### width

In [ ]:
parameters2[3]/parameters[3] # width

### area

In [ ]:
parameters2[1]/parameters[1] # area

In [ ]:
### x-axis limit

In [ ]:
parameters2[3]/parameters[3]*2  